# HF 电场梯度性质

这一节中，我们会简单回顾 HF 方法的电场梯度性质，这包括一阶梯度的偶极矩、与二阶梯度的极化率．这一节应当会对与电场的积分有所了解；同时对一阶 CP (Coupled-perturbed) 方程的使用有初步的了解．

In [ ]:
from pyscf import gto, scf, data, lib
import numpy as np

## HF 梯度性质参考文献

下面的参考书具有非常简洁有效但详实的，对 HF、ROHF 梯度以及 CI、MCSCF 等方法的推导与整理．

*  A New Dimension to Quantum Chemistry: Analytic Derivative Methods in Ab Initio Molecular Electronic Structure Theory
    * Yamaguchi, Y.; Goddard, J. D.; Osamura, Y.; Schaefer, H., Oxford University Press, 1994.
    * ISBN: [9780195070286](https://www.amazon.com/New-Dimension-Quantum-Chemistry-International/dp/0195070283)

## 准备工作

### 顶层函数计算 HF 梯度

In [ ]:
mol = gto.Mole()
mol.atom = """
O  1.0  0.0  0.0
H  1.0  1.0  0.0
H  1.0  0.0  1.0
"""
mol.basis = "6-31G"
mol.build()

In [ ]:
scf_eng = scf.RHF(mol)
energy_RHF = scf_eng.kernel()

### HF 重要中间矩阵

In [ ]:
nao = mol.nao
nmo = scf_eng.mo_energy.shape[0]
nelec = mol.nelectron
nocc = mol.nelec[0]
nvir = nmo - nocc

S = mol.intor('int1e_ovlp_sph')
T = mol.intor('int1e_kin_sph')
Vnuc = mol.intor('int1e_nuc_sph')
eri = mol.intor('int2e_sph')

C = scf_eng.mo_coeff
Co = C[:, :nocc]
Cv = C[:, nocc:]
e = scf_eng.mo_energy
eo = e[:nocc]
ev = e[nocc:]

D = scf_eng.make_rdm1()
F = scf_eng.get_fock()

J = scf_eng.get_j()
K = scf_eng.get_k()

## HF 偶极矩

### HF 偶极矩的 PySCF 程序计算

HF 偶极矩是与能量的一阶梯度有关的性质；关于偶极矩作为梯度性质的推导我们放在后续小节．前几小节中，我们先实现偶极矩的计算．

偶极矩的计算公式事实上在 Szabo (eq. 3.191) 中已经提及了．如果 $t \in \{ x, y, z \}$ 表示坐标分量，那么偶极矩作为三维空间的矢量 $\boldsymbol{\mu} = (\mu_x, \mu_y, \mu_z)$，其表达式为

\begin{equation}
\mu_{t} = - D_{\mu \nu} r_{\mu \nu}^t + Z_A X_{At}
\end{equation}

其中部分记号将在后面说明．PySCF 生成偶极矩的代码如下：

In [ ]:
scf_dip = scf_eng.dip_moment(unit="A.U.")

该结果可以与 Gaussian 计算结果相比较：

In [ ]:
np.allclose(scf_dip, [0., 0.7973507, 0.7973507])

### 偶极积分

偶极积分的相反数，$r_{\mu \nu}^t$ 是关于空间基的积分

\begin{equation}
r_{\mu \nu}^t = \langle \mu | t | \nu \rangle
\end{equation}

正因为在 PySCF 程序中，它是一个三角标的张量，因此我们会使用上述不太直观的记号．我们可以通过下述命令得到偶极积分相反数的维度信息：

In [ ]:
dip = mol.intor('int1e_r_sph')
dip.shape

如果使用显式的积分符号，并且注意到 $\mathrm{d} \boldsymbol{r} = \mathrm{d} x \, \mathrm{d} y \, \mathrm{d} z$，则以 $t = x$ 的情形为例，

\begin{equation}
\langle \mu | x | \nu \rangle = \iiint x \phi_{\mu} (\boldsymbol{r}) \phi_{\nu} (\boldsymbol{r}) \, \mathrm{d} x \, \mathrm{d} y \, \mathrm{d} z
\end{equation}

有必要对上述积分关于原点的选取问题作说明．我们知道，重叠积分 $S_{\mu \nu}$ 关于坐标系平动是不受干扰的：

\begin{align}
S_{\mu \nu} &= \int \phi_{\mu} (\boldsymbol{r} - \boldsymbol{r}_0) \phi_{\nu} (\boldsymbol{r} - \boldsymbol{r}_0) \, \mathrm{d} \boldsymbol{r} \\
&= \int \phi_{\mu} (\boldsymbol{r}) \phi_{\nu} (\boldsymbol{r}) \, \mathrm{d} (\boldsymbol{r} + \boldsymbol{r}_0) \\
&= \int \phi_{\mu} (\boldsymbol{r}) \phi_{\nu} (\boldsymbol{r}) \, \mathrm{d} \boldsymbol{r}
\end{align}

其中，上式的被积变量是 $\boldsymbol{r}$；$\boldsymbol{r}_0$ 则是确定的数值，表示坐标系平动；第三个等号利用了 $\boldsymbol{r}_0$ 为常量，因此其微分恒为零．

但是相同的情况对 $\langle \mu | x | \nu \rangle$ 并不成立．现在我们考虑坐标原点的积分 $\langle \mu | x | \nu \rangle$ 与坐标系平移 $\boldsymbol{r}_0 = (x_0, y_0, z_0)$ 后的积分值 $\langle \mu | x - x_0 | \nu \rangle$ 的关系：

\begin{align}
\langle \mu | x - x_0 | \nu \rangle &= \int (x - x_0) \phi_{\mu} (\boldsymbol{r} - \boldsymbol{r}_0) \phi_{\nu} (\boldsymbol{r} - \boldsymbol{r}_0) \, \mathrm{d} \boldsymbol{r} \\
&= \int x \phi_{\mu} (\boldsymbol{r} - \boldsymbol{r}_0) \phi_{\nu} (\boldsymbol{r} - \boldsymbol{r}_0) \, \mathrm{d} \boldsymbol{r} \\
&\quad - x_0 \int \phi_{\mu} (\boldsymbol{r} - \boldsymbol{r}_0) \phi_{\nu} (\boldsymbol{r} - \boldsymbol{r}_0) \, \mathrm{d} \boldsymbol{r} \\
&= \langle \mu | x | \nu \rangle - x_0 S_{\mu \nu}
\end{align}

因此，该积分与原点的选取值是有关的．

我们甚至可以简单地验证下述的结论．我们的坐标原点就是通常意义下的原点 $(0, 0, 0)$．对于上述的水分子，我们发现其 $x$ 轴分量上，三个原子都是 $x_0 = 1 \textrm{ Angstrom}$；它等价于，如果三个原子的 $x$ 坐标均为零，则我们求的 $x$ 方向的偶极积分即为 $\langle \mu | x - x_0 | \nu \rangle$．如果我们新设立一个分子 `mol_0`，它通过平移旧的分子 `mol` 得到，且使得其氧原子就在原点上，那么我们可以直接求得其偶极积分 $\langle \mu | x | \nu \rangle$．这两者的差应当正比于重叠积分 $S_{\mu \nu}$．

\begin{equation}
\langle \mu | x | \nu \rangle + x_0 S_{\mu \nu} = \langle \mu | x + x_0 | \nu \rangle = 0
\end{equation}

或者

\begin{equation}
\langle \mu | x + x_0 | \nu \rangle - \langle \mu | x | \nu \rangle = x_0 S_{\mu \nu}
\end{equation}

注意到我们如果采用原子单位，则 $x_0 \sim 0.529 \textrm{ Bohr}$．

In [ ]:
mol_0 = gto.Mole()
mol_0.atom = """
O  0.0  0.0  0.0
H  0.0  1.0  0.0
H  0.0  0.0  1.0
"""
mol_0.basis = "6-31G"
mol_0.build()

dip_0 = mol_0.intor('int1e_r_sph')

In [ ]:
np.allclose((dip[0] - dip_0[0]) * data.nist.BOHR, S)

### 偶极矩的计算

电子的偶极贡献可以通过偶极积分给出：

\begin{equation}
\mu_t^\textrm{elec} = - r_{\mu \nu}^t D_{\mu \nu}
\end{equation}

In [ ]:
dip_elec = - np.einsum("tuv, uv -> t", dip, D)
dip_elec

对比方才 PySCF 程序给出的 `scf_dip`，我们发现数值上的差别仍然很大．这是因为分子的偶极矩除了电子之外，还有原子核的核电荷贡献．其贡献可以通过下式给出：

\begin{equation}
\mu_t^\mathrm{nuc} = Z_A X_{At}
\end{equation}

其中，$Z_A$ 是指原子 $A$ 所带的电荷；$X_{At}$ 是指原子 $A$ 的位置在 $t$ 坐标分量上的值．上式的意义是，每一个偶极分量，即是各个原子在该分量的投影长度与原子的电荷的乘积．该数值可以立即由下述命令给出：

In [ ]:
dip_nuc = np.einsum("A, At -> t", mol.atom_charges(), mol.atom_coords())
dip_nuc

由此，总偶极矩可以通过上述两者的贡献加和获得．我们验证一下它是否与最初从 PySCF 获得的偶极矩的 [结果](#HF-偶极矩的-PySCF-程序计算) 一致：

In [ ]:
np.allclose(dip_elec + dip_nuc, scf_dip)

### HF 偶极矩的导出

现在我们会简单回顾 HF 方法下偶极矩的公式为何是上述的表达式．

偶极矩 $\boldsymbol{\mu}^\textsf{HF}$ 可以看作分子体系的 HF 能量对电场 $\boldsymbol{E}$ 响应的扰动量．这里的外加电场是空间上三维的向量．现在假设外加电场非常微弱，以至于不会影响分子构型与电子结构．

我们仍然像上面的计算一样，将分子的核与电子对偶极矩的贡献分离开看．由于核在化学计算中仍然当作经典的粒子，因此仍然可以套用经典的偶极矩公式

\begin{equation}
\boldsymbol{\mu}^\textrm{nuc} = \sum_A Z_A \boldsymbol{X}_A
\end{equation}

来考察各个原子核对参考点 (这里设定为原点) 的偶极贡献的和．

而电子对偶极矩的贡献并不显然．首先，我们写出含有电场微扰下的分子体系哈密顿算符：

\begin{equation}
\hat H^\mathrm{tot} = \hat H + \hat{\boldsymbol{\mu}} \cdot \boldsymbol{E}
\end{equation}

其中，$\hat H$ 是未被电场微扰的哈密顿算符；分子的电子云密度 $D_{\mu \nu}$ 就是通过该 $\hat H$ 确定．$\hat{\boldsymbol{\mu}} = (-x, -y, -z)$ 是对电子云的偶极矩算符，之所以会带负号是因为电子的电荷为 $-1$；之所以会是正比于 $(x, y, z)$ 的矢量，是因为偶极矩公式中本身具有着关于电荷与参考点间位移的矢量．

由此，偶极矩可以看作是微扰后体系的总能量对电场的偏导：

\begin{equation}
\boldsymbol{\mu}^\textrm{elec} = \nabla_{\boldsymbol{E}} \langle \Psi^\textsf{HF} | \hat H^\mathrm{tot} | \Psi^\textsf{HF} \rangle
\end{equation}

根据 HF 波函数本身具有的变分性质

\begin{align}
\langle \nabla_{\boldsymbol{E}} \Psi^\textsf{HF} | \hat H | \Psi^\textsf{HF} \rangle + \langle \Psi^\textsf{HF} | \hat H | \nabla_{\boldsymbol{E}} \Psi^\textsf{HF} \rangle = 0
\end{align}

以及因电场 $\boldsymbol{E} \rightarrow \boldsymbol{0}$ 的前提条件，从而得到的推论

\begin{equation}
\langle \nabla_{\boldsymbol{E}} \Psi^\textsf{HF} | \hat{\boldsymbol{\mu}} \cdot \boldsymbol{E} | \Psi^\textsf{HF} \rangle + \langle \Psi^\textsf{HF} | \hat{\boldsymbol{\mu}} \cdot \boldsymbol{E} | \nabla_{\boldsymbol{E}} \Psi^\textsf{HF} \rangle = 0
\end{equation}

从而，

\begin{align}
\langle \nabla_{\boldsymbol{E}} \Psi^\textsf{HF} | \hat H^\mathrm{tot} | \Psi^\textsf{HF} \rangle + \langle \Psi^\textsf{HF} | \hat H^\mathrm{tot} | \nabla_{\boldsymbol{E}} \Psi^\textsf{HF} \rangle = 0
\end{align}

进而能导出

\begin{equation}
\boldsymbol{\mu}^\textrm{elec} = \langle \Psi^\textsf{HF} | \hat{\boldsymbol{\mu}} | \Psi^\textsf{HF} \rangle
\end{equation}

即

\begin{equation}
\mu^\textrm{elec}_t = - r_{\mu \nu}^t D_{\mu \nu}
\end{equation}

这与 Yamaguchi (17.8) 使用 HF 能量对电场量的求导结论是一致的．

## HF 极化率

### HF 极化率的导出

这里我们需要先介绍极化率的公式导出方式．

极化率可以看作是对电场的二阶导数，也即偶极矩的电场一阶导数．由于偶极矩本身是电场一阶导数，为三维向量；而这里的二阶导数相当于算符的张量积，因此极化率是 $3 \times 3$ 的张量．

偶极矩可以分为原子核贡献与电子云贡献．我们指出，由于原子核贡献的偶极矩可以看作与微扰电场大小无关，因此其对电场的导数就为零．因此，我们只需要着眼于电子云的偶极矩导数的贡献即可．

电子云的偶极矩对极化率的贡献为

\begin{equation}
\boldsymbol{\alpha}^\textsf{HF} = \boldsymbol{\alpha}^\textrm{elec} = \nabla_{\boldsymbol{E}} \otimes \nabla_{\boldsymbol{E}} \langle \Psi^\textsf{HF} | \hat H^\mathrm{tot} | \Psi^\textsf{HF} \rangle
\end{equation}

上述的记号非常不直观；如果用我们熟悉的记号，则可以写为

\begin{equation}
\alpha_{tu}^\mathsf{HF} = \frac{\partial}{\partial E_t} \frac{\partial}{\partial E_s} \langle \Psi^\textsf{HF} | \hat H^\mathrm{tot} | \Psi^\textsf{HF} \rangle
\end{equation}

其中，$t, s$ 代表电场 $\boldsymbol{E}$ 的空间分量 $x, y, z$．

根据 Yamaguchi (17.54) 的结论，

\begin{equation}
\alpha_{ts}^\mathsf{HF} = - 4 U_{ai}^s h_{ai}^t = - 4 U_{ai}^s r_{ai}^t
\end{equation}

$U_{ai}^u$ 可以看作是电子云轨道在电场分量 $E_u$ 扰动下所产生的一阶响应矩阵，它需要通过 CP 方程求解获得；这些 CP 方程出处在 Yamaguchi (X.1-3):

\begin{equation}
(- \varepsilon_a + \varepsilon_i) U_{ai}^s - A_{ai, bj} U_{bj}^s = B_{ai}^s
\end{equation}

其中，

\begin{align}
A_{ai, bj} &= 4 (ai | bj) - (ab | ij) - (aj | bi) \\
B_{ai}^s &= h_{ai}^s = r_{ai}^s
\end{align}

注意到尽管原子核坐标对偶极矩有较大的贡献，但其对电场的偏导数为零；因此，在极化率中，我们无需考虑原子核坐标的贡献，而只需要考虑电子云的贡献即可．

### CP 方程的直接解

下面我们会用最为直接的方式解 CP 方程．我们稍微修改一下上述的 CP 方程，使之成为显式的线性方程：

\begin{align}
\delta_{ab} \delta_{ij} (- \varepsilon_a + \varepsilon_i) U_{ai}^s - A_{ai, bj} U_{bj}^s &= B_{ai}^s \\
\big( \delta_{ab} \delta_{ij} (- \varepsilon_a + \varepsilon_i) - A_{ai, bj} \big) U_{bj}^s &= r_{ai}^s 
\end{align}

我们令

\begin{equation}
A_{ai, bj}' = \big( \delta_{ab} \delta_{ij} (- \varepsilon_a + \varepsilon_i) - A_{ai, bj} \big)
\end{equation}

下面我们需要准备一些矩阵．首先，等式右边是分子轨道下的负值的偶极积分 $r_{ai}^s$：

\begin{equation}
r_{ai}^s = C_{\mu a} r_{\mu \nu}^s C_{\nu i}
\end{equation}

In [ ]:
dip_mo_ai = np.einsum("ua, suv, vi -> sai", Cv, dip, Co)

随后，等式左边的四脚标张量 $A_{ai, bj}$ 可以通过双电子积分给出．但我们可以用相对低效一些的方法获得：我们先获得分子轨道下的化学约定的双电子积分

\begin{equation}
(pq | rs) = C_{\mu p} C_{\nu q} (\mu \nu | \kappa \lambda) C_{\kappa r} C_{\lambda s}
\end{equation}

In [ ]:
eri_mo = np.einsum("up, vq, uvkl, kr, ls -> pqrs", C, C, eri, C, C, optimize=True)

通过交换角标顺序，相加减之后，我们就可以获得 $A_{pq, rs}$ 矩阵；随后依据占据-非占轨道数，截取 $A_{pq, rs}$ 到 $A_{ai, bj}$：

In [ ]:
A = 4 * eri_mo - eri_mo.transpose(0, 2, 1, 3) - eri_mo.transpose(0, 2, 3, 1)
A = A[nocc:, :nocc, nocc:, :nocc]

<div class="alert alert-info">

**提示**

注意到 $(pq|rs)$ 到 $(ps|qr)$ 的转换是 `eri_mo.transpose(0, 2, 3, 1)` 而不是 `eri_mo.transpose(0, 3, 1, 2)`．如果我们现在记 $(ps|qr) = (01|23)$，那么 $(pq|rs) = (02|31)$．

</div>

我们定义下述二维张量

\begin{equation}
D_i^a = - \varepsilon_a + \varepsilon_i
\end{equation}

In [ ]:
D_ia = lib.direct_sum("i - a -> ia", eo, ev)

那么，$A_{ai, bj}' = \delta_{ab} \delta_{ij} D_i^a - A_{ai, bj}$ 就可以用代码表示为

In [ ]:
A_prime = np.einsum("ab, ij, ia -> aibj", np.eye(nvir), np.eye(nocc), D_ia) - A

现在我们可以求变换矩阵了．如果我们将 $A_{ai, bj}'$ 看作二维矩阵，$U_{bj}^s$ 与 $r_{ai}^s$ 看作一维向量，那么 $U_{bj}^s$ 可以直接通过矩阵求逆获得：

\begin{equation}
U_{bj}^s = A'{}^{-1}_{ai, bj} r_{ai}^s
\end{equation}

为此，我们需要在代码中加入角标压平的操作，或者说使用变换张量维度的操作．

In [ ]:
U = np.einsum("xy, sx -> sy", np.linalg.inv(A_prime.reshape(nvir * nocc, nvir * nocc)), dip_mo_ai.reshape(3, -1))
U.shape = (3, nvir, nocc)

最终，我们代入偶极矩公式，就可以得到偶极矩值了：

\begin{equation}
\alpha_{ts}^\mathsf{HF} = - 4 U_{ai}^s h_{ai}^t = - 4 U_{ai}^s r_{ai}^t
\end{equation}

In [ ]:
alpha_hf = -4 * np.einsum("sai, tai -> ts", U.reshape(3, nvir, nocc), dip_mo_ai)
alpha_hf

我们可以通过与 Gaussian 的输出核验上述结果是否正确．我们知道，Gaussian 输出的极化率的 [排列方式](http://gaussian.com/polar/) 是 $(xx, xy, yy, xz, yz, zz)$，因此，下述从 `alpha_hf` 中抽提出来的结果应该与 Gaussian 输出结果一致 ([Gaussian 输入卡](include/HF-dipole.gjf) 与 [输出文件](include/HF-dipole.out))：

In [ ]:
np.allclose([alpha_hf[tup] for tup in [(0,0), (0,1), (1,1), (0,2), (1,2), (2,2)]],
            [1.32195961E+00, 1.83494037E-14, 6.56963343E+00, 1.76113980E-14, -5.16993803E-01, 6.56963342E+00])

### CP 方程的迭代求解

事实上，由于四维张量的求逆从复杂度的角度上非常困难，因此需要寻找迭代求解的方法．最简单的迭代求解是自洽求解．下面我们简单地实现这个过程．

回顾 CP 方程：

\begin{equation}
D_i^a U_{ai}^s - A_{ai, bj} U_{bj}^s = r_{ai}^s
\end{equation}

其中一种简单的迭代方案是

\begin{equation}
U_{ai}^s = (D_i^a)^{-1} (r_{ai}^s + A_{ai, bj} U_{bj}^s)
\end{equation}

我们定义上述收敛条件是 $U_{ai}^s$ [Frobenius 范数](https://en.wikipedia.org/wiki/Matrix_norm) 小于 $10^{-10}$，输出矩阵命名为 `U_iter`，且初猜定为零矩阵，那么收敛过程如下：

In [ ]:
U_iter = np.zeros((3, nvir, nocc))
U_iter_old = U_iter.copy() + 1.
iter_count = 0
while (np.linalg.norm(U_iter - U_iter_old) > 1.e-10):
    iter_count += 1
    U_iter_old = U_iter.copy()
    U_iter = (1 / D_ia.T) * (dip_mo_ai + np.einsum("aibj, sbj -> sai", A, U_iter))
print("Total iteration times: ", iter_count)

我们很容易地验证，上述迭代求得的变换矩阵与之前对 $A_{ai, bj}$ 求逆得到的变换矩阵是相同的：

In [ ]:
np.allclose(U, U_iter)

### 避免四脚标张量 $A_{ai, bj}$

事实上，四脚标张量 $A_{ai, bj}$ 是可以避免生成的．我们知道，在计算 HF 能量时，我们通过代入 SCF 的密度矩阵来求得库伦与交换积分 $J_{\mu \nu} [\mathbf{D}]$ 与 $K_{\mu \nu} [\mathbf{D}]$．

\begin{align}
J_{\mu \nu} [\mathbf{D}] &= (\mu \nu | \kappa \lambda) D_{\kappa \lambda} \\
K_{\mu \nu} [\mathbf{D}] &= (\mu \kappa | \nu \lambda) D_{\kappa \lambda}
\end{align}

在这里，$A_{ai, bj}$ 张量的三个项在与 $U_{bj}^s$ 相乘之后，可以写成 4 份库伦积分与 2 份交换积分的总和：

\begin{align}
(ai|bj) U_{bj}^s = C_{\mu a} C_{\nu i} (\mu \nu | \kappa \lambda) C_{\kappa b} U_{bj}^s C_{\lambda j} &= C_{\mu a} J_{\mu \nu} [\mathbf{D}] C_{\nu i} \\
(ab|ij) U_{bj}^s = C_{\mu a} C_{\nu i} (\mu \kappa | \nu \lambda) C_{\kappa b} U_{bj}^s C_{\lambda j} &= C_{\mu a} K_{\mu \nu} [\mathbf{D}] C_{\nu i} \\
(aj|bi) U_{bj}^s = C_{\mu a} C_{\nu i} (\mu \kappa | \lambda \nu) C_{\lambda b} U_{bj}^s C_{\kappa j} &= C_{\mu a} K_{\mu \nu} [\mathbf{D}^\mathrm{T}] C_{\nu i}
\end{align}

其中，上面的 $\mathbf{D}$ 即 $D_{\kappa \lambda} = C_{\kappa b} U_{bj}^s C_{\lambda j}$ 并不是 SCF 的密度矩阵，而是变换矩阵 $U_{bj}^s$ 原子轨道化后得到的广义密度矩阵．

利用 HF 方法中，Fock 矩阵势能贡献可以简写为 $V_{\mu \nu}^\textrm{JK} [\mathbf{D}] = J_{\mu \nu} [\mathbf{D}] C_{\nu i} - \frac 1 2 K_{\mu \nu} [\mathbf{D}] C_{\nu i}$，以及库伦积分本身的对称性，可以将 $A_{ai, bj}$ 记为

\begin{align}
A_{ai, bj} &= 4 C_{\mu a} J_{\mu \nu} [\mathbf{D}] C_{\nu i} - K_{\mu \nu} [\mathbf{D}] C_{\nu i} - C_{\mu a} K_{\mu \nu} [\mathbf{D}^\mathrm{T}] C_{\nu i} \\
&= C_{\mu a} (2 J_{\mu \nu} [\mathbf{D}] - K_{\mu \nu} [\mathbf{D}]) C_{\nu i} + C_{\mu a} (2 J_{\mu \nu} [\mathbf{D}^\mathrm{T}] - K_{\mu \nu} [\mathbf{D}^\mathrm{T}]) C_{\nu i} \\
&= C_{\mu a} (2 V_{\mu \nu}^\textrm{JK} [\mathbf{D}] + 2 V_{\mu \nu}^\textrm{JK} [\mathbf{D}^\mathrm{T}]) \\
&= 2 C_{\mu a} V_{\mu \nu}^\textrm{JK} [\mathbf{D} + \mathbf{D}^\mathrm{T}] C_{\nu i}
\end{align}

下面的代码中，我们通过函数 `ax` 来辅助实现 $A_{ai, bj}$ 与 $U_{bj}^s$ 的乘积．可以验证，它与直接 CP 方法中获得的 $A_{ai, bj} U_{bj}^s$ 的结果是一样的．尽管我们只使用了收敛完毕的 $U_{bj}^s$ 验证两种方法得到的 $A_{ai, bj} U_{bj}^s$ 相同，但这也适用于其它任何维度上符合的矩阵．

In [ ]:
def ax(x):
    x = x.reshape((nvir, nocc))
    dx = Cv @ x @ Co.T
    v = Cv.T @ scf_eng.get_veff(mol, dx + dx.T) @ Co
    return 2 * v
np.allclose([ax(U[i]) for i in range(3)], np.einsum("aibj, sbj -> sai", A, U))

因此，我们可以利用上面的 `ax` 函数重新写 CP 方程的迭代过程．可以注意到，这次的代码中，没有涉及到显式的双电子积分，也没有很复杂的张量转置或者求逆、变换矩阵维度的操作．因此，尽管 CP 方程的形式看起来比较复杂，但只考虑最基础的实现，到代码层面上，还是很容易解决的．

In [ ]:
U_iter = np.zeros((3, nvir, nocc))
U_iter_old = U_iter.copy() + 1.
iter_count = 0
while (np.linalg.norm(U_iter - U_iter_old) > 1.e-10):
    iter_count += 1
    U_iter_old = U_iter.copy()
    U_iter = (1 / D_ia.T) * (dip_mo_ai + [ax(U_iter[i]) for i in range(3)])
print("Total iteration times: ", iter_count)

在 PySCF 中，还设计了一种解决 CP 方程的函数 `scf.cphf.solve`．它可以代替上述的代码块以实现 CP 方程的计算．我们将结果记为 `U_easy`；它在一定精度内，与上面精确计算得到的转换矩阵 $U_{ai}^s$ 的结果是相同的．

In [ ]:
U_easy = np.array([scf.cphf.solve(ax, e, scf_eng.mo_occ, dip_mo_ai[i], max_cycle=100)[0] for i in range(0, 3)])
np.allclose(U_easy, U, atol = 1.e-7)